### Quantitative Metrics for Model Evaluation

**Objective:**

  - Calculate key evaluation metrics to assess model prediction performance on the test set.
  - Understand RMSE (Root Mean Squared Error) and MAE (Mean Absolute Error) as primary metrics for RUL prediction.
  - Implement metric calculations using the test dataset and previously trained model predictions.

#### 1. Imports and Setup

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
import matplotlib.pyplot as plt

# Suppress warnings for clearer output
import warnings
warnings.filterwarnings('ignore')

c:\Users\roji2\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\roji2\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\roji2\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framewo

#### 2. Explanation: Evaluation Metrics for Regression (RUL Prediction)


2.1 Root Mean Squared Error (RMSE):
- RMSE measures the square root of the average squared differences between predicted and actual values.
- It penalizes larger errors more heavily, making it sensitive to outliers.
- Lower RMSE indicates better model performance.

Formula: RMSE = sqrt( (1/N) * Σ(y_true_i - y_pred_i)^2 )

2.2 Mean Absolute Error (MAE):
- MAE measures the average absolute differences between predicted and actual values.
- It provides a straightforward interpretation of average error magnitude.
- Less sensitive to large errors compared to RMSE.

Formula: MAE = (1/N) * Σ|y_true_i - y_pred_i|


#### 3. Load Test Data and Previous Model

In [14]:
x_data = np.load(
    r"D:\ProgonsAI\milestone_2\week_4\Day_16\rolling_window_sequences.npy",
    mmap_mode="r"
)

metadata = pd.read_csv(
    r"D:\ProgonsAI\milestone_2\week_4\Day_16\sequence_metadata_with_RUL.csv"
)

print("Full dataset shapes:")
print("x_data:", x_data.shape)
print("metadata rows:", metadata.shape)

# Extract TRUE test set = last 30 cycles of each engine
N = 30
metadata["is_test"] = metadata["cycle"] >= (metadata["max_cycle_new"] - N)

x_test = x_data[metadata["is_test"].values]
y_test = metadata.loc[metadata["is_test"], "RUL"].values

print("\nTest set shapes:")
print("x_test:", x_test.shape)
print("y_test:", y_test.shape)

# Load trained model
model = tf.keras.models.load_model(
    r"D:\ProgonsAI\milestone_2\week_4\Day_16\best_model.keras"
)

print("\nModel loaded successfully!")

Full dataset shapes:
x_data: (152559, 30, 66)
metadata rows: (152559, 6)

Test set shapes:
x_test: (9490, 30, 66)
y_test: (9490,)

Model loaded successfully!


#### 4. Generate Predictions on Test Set

In [15]:
#### 4. Generate Predictions on Test Set

batch_size = 500
num_samples = x_test.shape[0]
y_pred_list = []

for start in range(0, num_samples, batch_size):
    end = start + batch_size
    batch = x_test[start:end]
    batch_pred = model.predict(batch, verbose=0)
    y_pred_list.append(batch_pred)

# Combine all predictions
y_pred = np.concatenate(y_pred_list).flatten()

print("Final prediction shape:", y_pred.shape)


Final prediction shape: (9490,)


#### 5. Calculate RMSE and MAE

In [16]:
#### 5. Calculate RMSE and MAE

from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"Correct RMSE: {rmse:.4f}")
print(f"Correct MAE : {mae:.4f}")


Correct RMSE: 90.8977
Correct MAE : 87.3143


#### 6. Visualize Predicted vs Actual RUL